In [1]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
from bs4 import BeautifulSoup

# スプレッドシートに書き込む

# 認証情報ファイルのパス
CREDENTIAL_FILE_PATH = '/Users/masaki/Desktop/【json】/gspread-test-408312-2072e3277dcd.json'

# スプレッドシートのキー
SPREADSHEET_KEY = '1mF2nBRBPxeYog7YnBFy-1QNxDmjwUl5O2ZhqB6y7_Qw'

# 認証情報の取得
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIAL_FILE_PATH)

# スプレッドシートのオープン
gc = gspread.authorize(credentials)
sh = gc.open_by_key(SPREADSHEET_KEY)

# ワークシートの取得
worksheet = sh.worksheet('Billboard')

# Billboardのウェブページからデータを取得
url = 'https://www.billboard.com/charts/hot-100/'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

# チャートの結果を取得
chart_results = soup.find_all('div', attrs={'class': 'o-chart-results-list-row-container'})

# データを格納するリスト
data_list = []

for chart_result in chart_results:
    # 順位を取得
    rank = chart_result.find_all('span', attrs={'class': 'c-label'})[0].text.strip()
    rank = int(rank)

    # タイトルを取得
    title = chart_result.find_all('h3', attrs={'class': 'c-title'})[0].text.strip()

    # アーティスト名を取得
    artist_name = chart_result.find_all('span', class_='a-no-trucate')[0].text.strip()

    # 先週の順位を取得
    lastWeek = chart_result.find_all('span', class_='c-label')[2].text.strip()

    # データをリストに追加
    data_list.append([rank, title, artist_name, lastWeek])

# 空白行の数
num_blank_rows = 5

# 空白行を追加するための空のリストを作成
blank_rows = [[''] * len(data_list[0])] * num_blank_rows

# 空白行とデータを結合
data_with_blanks = blank_rows + data_list

# A2 から書き込む準備
start_row = 2  # A2
start_col = 1  # A列
total_rows = len(data_with_blanks)
total_cols = len(data_with_blanks[0])  # 例: 4列

# 終了セルの列番号を文字に変換（例：D）
end_col_letter = chr(ord('A') + total_cols - 1)
end_row_number = start_row + total_rows - 1

range_str = f'A{start_row}:{end_col_letter}{end_row_number}'

# A2から空白行込みのデータを一括更新
worksheet.update(range_str, data_with_blanks)

print("新しいデータがスプレッドシートにA2から書き込まれました。")


/var/folders/7g/x7vmxdmx1p556h61blyys4bm0000gp/T/ipykernel_71920/3931189169.py:77: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(range_str, data_with_blanks)


新しいデータがスプレッドシートにA2から書き込まれました。
